In [2]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 5: Binary Boarding ---</h2><p>You board your plane only to discover a new problem: you dropped your boarding pass! You aren't sure which seat is yours, and all of the flight attendants are busy with the flood of people that suddenly made it through passport control.</p>
<p>You write a <span title="No problem!">quick program</span> to use your phone's camera to scan all of the nearby boarding passes (your puzzle input); perhaps you can find your seat through process of elimination.</p>
<p>Instead of <a target="_blank" href="https://www.youtube.com/watch?v=oAHbLRjF0vo">zones or groups</a>, this airline uses <em>binary space partitioning</em> to seat people. A seat might be specified like <code>FBFBBFFRLR</code>, where <code>F</code> means "front", <code>B</code> means "back", <code>L</code> means "left", and <code>R</code> means "right".</p>
<p>The first 7 characters will either be <code>F</code> or <code>B</code>; these specify exactly one of the <em>128 rows</em> on the plane (numbered <code>0</code> through <code>127</code>). Each letter tells you which half of a region the given seat is in. Start with the whole list of rows; the first letter indicates whether the seat is in the <em>front</em> (<code>0</code> through <code>63</code>) or the <em>back</em> (<code>64</code> through <code>127</code>). The next letter indicates which half of that region the seat is in, and so on until you're left with exactly one row.</p>
<p>For example, consider just the first seven characters of <code>FBFBBFFRLR</code>:</p>
<ul>
<li>Start by considering the whole range, rows <code>0</code> through <code>127</code>.</li>
<li><code>F</code> means to take the <em>lower half</em>, keeping rows <code>0</code> through <code>63</code>.</li>
<li><code>B</code> means to take the <em>upper half</em>, keeping rows <code>32</code> through <code>63</code>.</li>
<li><code>F</code> means to take the <em>lower half</em>, keeping rows <code>32</code> through <code>47</code>.</li>
<li><code>B</code> means to take the <em>upper half</em>, keeping rows <code>40</code> through <code>47</code>.</li>
<li><code>B</code> keeps rows <code>44</code> through <code>47</code>.</li>
<li><code>F</code> keeps rows <code>44</code> through <code>45</code>.</li>
<li>The final <code>F</code> keeps the lower of the two, <em>row <code>44</code></em>.</li>
</ul>
<p>The last three characters will be either <code>L</code> or <code>R</code>; these specify exactly one of the <em>8 columns</em> of seats on the plane (numbered <code>0</code> through <code>7</code>). The same process as above proceeds again, this time with only three steps.  <code>L</code> means to keep the <em>lower half</em>, while <code>R</code> means to keep the <em>upper half</em>.</p>
<p>For example, consider just the last 3 characters of <code>FBFBBFFRLR</code>:</p>
<ul>
<li>Start by considering the whole range, columns <code>0</code> through <code>7</code>.</li>
<li><code>R</code> means to take the <em>upper half</em>, keeping columns <code>4</code> through <code>7</code>.</li>
<li><code>L</code> means to take the <em>lower half</em>, keeping columns <code>4</code> through <code>5</code>.</li>
<li>The final <code>R</code> keeps the upper of the two, <em>column <code>5</code></em>.</li>
</ul>
<p>So, decoding <code>FBFBBFFRLR</code> reveals that it is the seat at <em>row <code>44</code>, column <code>5</code></em>.</p>
<p>Every seat also has a unique <em>seat ID</em>: multiply the row by 8, then add the column. In this example, the seat has ID <code>44 * 8 + 5 = <em>357</em></code>.</p>
<p>Here are some other boarding passes:</p>
<ul>
<li><code>BFFFBBFRRR</code>: row <code>70</code>, column <code>7</code>, seat ID <code>567</code>.</li>
<li><code>FFFBBBFRRR</code>: row <code>14</code>, column <code>7</code>, seat ID <code>119</code>.</li>
<li><code>BBFFBBFRLL</code>: row <code>102</code>, column <code>4</code>, seat ID <code>820</code>.</li>
</ul>
<p>As a sanity check, look through your list of boarding passes. <em>What is the highest seat ID on a boarding pass?</em></p>
</article>


In [13]:
tests = [
    {
        "name": "Example 1: FBFBBFFRLR: row 44, column 5, seat ID 357",
        "boarding_pass": "FBFBBFFRLR",
        "expected": 357,
    },
    {
        "name": "Example 2: BFFFBBFRRR: row 70, column 7, seat ID 567",
        "boarding_pass": "BFFFBBFRRR",
        "expected": 567,
    },
    {
        "name": "Example 3: FFFBBBFRRR: row 14, column 7, seat ID 119",
        "boarding_pass": "FFFBBBFRRR",
        "expected": 119,
    },
    {
        "name": "Example 4: BBFFBBFRLL: row 102, column 4, seat ID 820",
        "boarding_pass": "BBFFBBFRLL",
        "expected": 820,
    },
]


def seat_id(boarding_pass: str) -> int:
    row = int(boarding_pass[:7].replace("F", "0").replace("B", "1"), 2)
    col = int(boarding_pass[7:].replace("L", "0").replace("R", "1"), 2)
    return 8 * row + col


run_tests_params(seat_id, tests)

example = """
FFFBBBFRRR
BBFFBBFRLL
BFFFBBFRRR
"""


def highest_seat_id(s: str) -> int:
    return max(map(seat_id, s.strip().splitlines()))


assert highest_seat_id(example) == 820


Test Example 1: FBFBBFFRLR: row 44, column 5, seat ID 357 passed, for seat_id.
Test Example 2: BFFFBBFRRR: row 70, column 7, seat ID 567 passed, for seat_id.
Test Example 3: FFFBBBFRRR: row 14, column 7, seat ID 119 passed, for seat_id.
Test Example 4: BBFFBBFRLL: row 102, column 4, seat ID 820 passed, for seat_id.
Success


In [14]:
with open("../input/day5.txt") as f:
    puzzle = f.read()

print(f"Part I: {highest_seat_id(puzzle)}")

Part I: 818


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>818</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p><em>Ding!</em> The "fasten seat belt" signs have turned on. Time to find your seat.</p>
<p>It's a completely full flight, so your seat should be the only missing boarding pass in your list.  However, there's a catch: some of the seats at the very front and back of the plane don't exist on this aircraft, so they'll be missing from your list as well.</p>
<p>Your seat wasn't at the very front or back, though; the seats with IDs +1 and -1 from yours will be in your list.</p>
<p><em>What is the ID of your seat?</em></p>
</article>

</main>


In [19]:
def my_seat_id(s: str) -> int:
    seat_ids = sorted(seat_id(bp) for bp in s.strip().splitlines())
    prev_id = seat_ids[0]
    for i in range(1, len(seat_ids)):
        if seat_ids[i] - prev_id > 1:
            return prev_id + 1
        prev_id = seat_ids[i]


print(f"Part II: {my_seat_id(puzzle)}")

Part II: 559


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>559</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
